In [59]:
# Simple CNN imports

import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow.keras import datasets, layers, models

In [1]:
# Xception imports

import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D
# import tensorflow.contrib.eager as tfe
from matplotlib import pyplot as plt
import numpy as np
import os
import time
from PIL import Image

# tf.enable_eager_execution()

In [2]:
train_img_path = 'data/train/'
test_img_path = 'data/test/'

train_set = []
for label in os.listdir(train_img_path):
    if label == ".DS_Store":
        continue
    for img in os.listdir(os.path.join(train_img_path, label)):
        if img == ".DS_Store":
            continue
        train_set.append((os.path.join(train_img_path, label, img), int(label)))

test_set = []
for label in os.listdir(test_img_path):
    if label == ".DS_Store":
        continue
    for img in os.listdir(os.path.join(test_img_path, label)):
        if img == ".DS_Store":
            continue
        test_set.append((os.path.join(test_img_path, label, img), int(label)))

In [3]:
train_imgs = []
train_labels = []
start_time = time.time()
for img, label in train_set:
    img = np.asarray(Image.open(img))
    train_imgs.append(img)
    train_labels.append(label)
print("--- %s seconds ---" % (time.time() - start_time))  

test_imgs = []
test_labels = []
start_time = time.time()
for img, label in test_set:
    img = np.asarray(Image.open(img))
    test_imgs.append(img)
    test_labels.append(label)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 57.42507815361023 seconds ---
--- 13.50636076927185 seconds ---


## Simple CNN

In [60]:
model = models.Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(22, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(np.array(train_imgs), np.array(train_labels), epochs=10, shuffle=True, validation_data=(np.array(test_imgs), np.array(test_labels)))

Train on 78240 samples, validate on 19560 samples
Epoch 1/10
78240/78240 [==============================] - 1933s 25ms/sample - loss: 2.9659 - accuracy: 0.1053 - val_loss: 2.7563 - val_accuracy: 0.1634
Epoch 2/10
78240/78240 [==============================] - 1738s 22ms/sample - loss: 2.3188 - accuracy: 0.3015 - val_loss: 1.8407 - val_accuracy: 0.4529
Epoch 3/10
78240/78240 [==============================] - 1724s 22ms/sample - loss: 1.5052 - accuracy: 0.5446 - val_loss: 1.4420 - val_accuracy: 0.5670
Epoch 4/10
78240/78240 [==============================] - 1727s 22ms/sample - loss: 1.0611 - accuracy: 0.6769 - val_loss: 1.3837 - val_accuracy: 0.6036
Epoch 5/10
78240/78240 [==============================] - 1740s 22ms/sample - loss: 0.7571 - accuracy: 0.7639 - val_loss: 1.5120 - val_accuracy: 0.5972
Epoch 6/10
78240/78240 [==============================] - 1730s 22ms/sample - loss: 0.5430 - accuracy: 0.8283 - val_loss: 1.6307 - val_accuracy: 0.6198
Epoch 7/10
78240/78240 [==============

## CNN with Xception pretrained model:
In this model we use a pretrained model to find the features, and then add a 3 layer CNN on top of that model as a classifier

In [22]:
class XceptionBottleneck(tf.keras.Model):
    
    def __init__(self):
        super(XceptionBottleneck, self).__init__()
        
        # Define xception layer (include_top=False and use imagenet weights), 
        # see: https://keras.io/applications/#models-for-image-classification-with-weights-trained-on-imagenet
        self.xception_layers = Xception(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
        
        # Define pooling GlobalAveragePooling2D 
        # see: https://keras.io/layers/pooling/  
        self.pooling_layer = GlobalAveragePooling2D()
        
    def call(self, inputs):
        result = self.xception_layers(inputs)
        result = self.pooling_layer(result)
        return result

In [23]:
def cache_bottleneck_layers(numpy_data, batch_size, device):
    
    bottle_necks = []
    dataset = tf.data.Dataset.from_tensor_slices(numpy_data).batch(batch_size)
    n_samples = numpy_data.shape[0]

    with tf.device(device):
        xception_out = XceptionBottleneck()
        for batch_num, image in enumerate(dataset):
            print('\rComputing bottle neck layers... batch {} of {}'.format(batch_num+1, n_samples//batch_size), end="")
            
            result = xception_out(image)
            result = result.numpy()
            bottle_necks.append(result)
            
    return np.vstack(bottle_necks)

In [39]:
start_time = time.time()

# device = "gpu:0" if tfe.num_gpus() else "cpu:0"
device = "cpu:0"
bottle_necks = cache_bottleneck_layers(np.array(train_imgs, dtype=np.float32), batch_size=50, device=device)

print("--- %s seconds ---" % (time.time() - start_time)) 

Computing bottle neck layers... batch 1565 of 1564--- 2872.5276679992676 seconds ---


In [40]:
cache_path = "./cache/"
fname = 'bottle_neck_train.npz'
train_save_path = os.path.join(cache_path, fname)

if not os.path.isdir(cache_path): 
    os.mkdir(cache_path)
    
np.savez(train_save_path, bottle_necks=bottle_necks, labels=np.array(train_labels))

In [41]:
start_time = time.time()

# device = "gpu:0" if tfe.num_gpus() else "cpu:0"
device = "cpu:0"
bottle_necks = cache_bottleneck_layers(np.array(test_imgs, dtype=np.float32), batch_size=50, device=device)

print("--- %s seconds ---" % (time.time() - start_time)) 

Computing bottle neck layers... batch 392 of 391--- 638.8322191238403 seconds ---


In [42]:
cache_path = "./cache/"
fname = 'bottle_neck_test.npz'
test_save_path = os.path.join(cache_path, fname)

if not os.path.isdir(cache_path): 
    os.mkdir(cache_path)
    
np.savez(test_save_path, bottle_necks=bottle_necks, labels=np.array(test_labels))

In [43]:
class XceptionClassifier(tf.keras.Model):
    
    def __init__(self, n_classes):
        super(XceptionClassifier, self).__init__()
        # Define the layer(s) you would like to use for your classifier
        self.dense_layer_1 = tf.keras.layers.Dense(units=20*n_classes, activation=tf.keras.activations.relu)
        self.dense_layer_2 = tf.keras.layers.Dense(units=5*n_classes, activation=tf.keras.activations.relu)
        self.dense_layer_3 = tf.keras.layers.Dense(units=n_classes)
        
    def call(self, inputs):
        # Set this up appropriately, will depend on how many layers you choose
        result = self.dense_layer_1(inputs)
        result = self.dense_layer_2(result)
        result = self.dense_layer_3(result)
        return result

In [44]:
data = np.load(train_save_path)
train_bottle_necks, train_labels = data['bottle_necks'],  data['labels']

data = np.load(test_save_path)
test_bottle_necks, test_labels = data['bottle_necks'],  data['labels']

In [45]:
batch_size = 16        # You will play around with this 
n_epochs = 200         # Choose num epochs based on when you think the parameters have converged
learning_rate = 0.00001 # You will try different learning rates

train_loss_history = []

In [46]:
train_images_dataset = tf.data.Dataset.from_tensor_slices(train_bottle_necks)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(train_labels)
train_dataset = tf.data.Dataset.zip((train_images_dataset, train_labels_dataset))
train_dataset = train_dataset.batch(batch_size).shuffle(buffer_size=train_bottle_necks.shape[0])

In [48]:
x_classifier = XceptionClassifier(n_classes=22)
optimizer = tf.optimizers.Adam(learning_rate) 

In [52]:
start_time = time.time()
with tf.device(device):
    for epoch in range(n_epochs):
        for batch, (images, labels) in enumerate(train_dataset):

            with tf.GradientTape() as tape:
                # Compute logits
                logits = x_classifier(images)
                xe_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
                
            train_loss_history.append(xe_loss.numpy())
            # Compute gradient and apply gradients
            grads = tape.gradient(xe_loss, x_classifier.variables)
            optimizer.apply_gradients(zip(grads, x_classifier.variables))
#                                       , global_step=tf.compat.v1.train.get_or_create_global_step())
            
            if batch % 10 == 0:
                print('\rEpoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, train_loss_history[-1]), end='')
print("--- %s seconds ---" % (time.time() - start_time)) 

Epoch: 199, Batch: 4880, Loss: 0.1278851181268692737--- 21690.22667312622 seconds ---


In [57]:
## computing the train accuracy for first model
logits = x_classifier(tf.constant(train_bottle_necks))
y_pred = tf.nn.softmax(logits).numpy()
train_pred = np.argmax(y_pred, axis=1)
train_accuracy = np.sum(train_pred == train_labels) / len(train_pred)

## computing the validation accuracy for first model
logits = x_classifier(tf.constant(test_bottle_necks))
y_pred = tf.nn.softmax(logits).numpy()
test_pred = np.argmax(y_pred, axis=1)
test_accuracy = np.sum(test_pred == test_labels) / len(test_pred)

print(train_accuracy, test_accuracy)
# .... complete parts (e,f)

0.9116180981595092 0.5053680981595092


In [55]:
x_classifier_100 = XceptionClassifier(n_classes=20)
optimizer = tf.optimizers.Adam(learning_rate)
n_epochs = 100

start_time = time.time()
with tf.device(device):
    for epoch in range(n_epochs):
        for batch, (images, labels) in enumerate(train_dataset):

            with tf.GradientTape() as tape:
                # Compute logits
                logits = x_classifier_100(images)
                xe_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
                
            train_loss_history.append(xe_loss.numpy())
            # Compute gradient and apply gradients
            grads = tape.gradient(xe_loss, x_classifier_100.variables)
            optimizer.apply_gradients(zip(grads, x_classifier_100.variables))
#                                       , global_step=tf.compat.v1.train.get_or_create_global_step())
            
            if batch % 10 == 0:
                print('\rEpoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, train_loss_history[-1]), end='')
print("--- %s seconds ---" % (time.time() - start_time)) 

Epoch: 99, Batch: 4880, Loss: 0.908546507358551--- 4780.221822977066 seconds ---


In [56]:
## computing the train accuracy for first model
logits = x_classifier_100(tf.constant(train_bottle_necks))
y_pred = tf.nn.softmax(logits).numpy()
train_pred = np.argmax(y_pred, axis=1)
train_accuracy = np.sum(train_pred == train_labels) / len(train_pred)

## computing the validation accuracy for first model
logits = x_classifier_100(tf.constant(test_bottle_necks))
y_pred = tf.nn.softmax(logits).numpy()
test_pred = np.argmax(y_pred, axis=1)
test_accuracy = np.sum(test_pred == test_labels) / len(test_pred)

print(train_accuracy, test_accuracy)
# .... complete parts (e,f)

0.7262525562372188 0.4947852760736196
